In [32]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
############################### imports
from sklearn.metrics import hamming_loss
from sklearn.metrics import jaccard_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn import metrics
from sklearn.svm import LinearSVC
import transformers
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
import seaborn as sns
import shutil, sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import torch
import os

In [33]:

import warnings

warnings.filterwarnings("ignore")


# Read CSV files to get questions and tags
df_questions = pd.read_csv("./Questions.csv", encoding="ISO-8859-1")
df_tags = pd.read_csv("./Tags.csv", encoding="ISO-8859-1", dtype={'Tag': str})


df_questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in one statement,"<p>I've written a database generation script in <a href=""http://en.wikipedia.org/wiki/SQL"">SQL</a> and want to execute it in my <a href=""http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime"">Adobe AIR</a> application:</p>\n\n<pre><code>Create Table tRole (\n roleID integer Primary Key\n ,roleName varchar(40)\n);\nCreate Table tFile (\n fileID integer Primary Key\n ,fileName varchar(50)\n ,fileDescription varchar(500)\n ,thumbnailID integer\n ,fileFormatID integer\n ,categoryID integer\n ,isFavorite boolean\n ,dateAdded date\n ,globalAccessCount integer\n ,lastAccessTime date\n ,downloadComplete boolean\n ,isNew boolean\n ,isSpotlight boolean\n ,duration varchar(30)\n);\nCreate Table tCategory (\n categoryID integer Primary Key\n ,categoryName varchar(50)\n ,parent_categoryID integer\n);\n...\n</code></pre>\n\n<p>I execute this in Adobe AIR using the following methods:</p>\n\n<pre><code>public static function RunSqlFromFile(fileName:String):void {\n var file:File = File.applicationDirectory.resolvePath(fileName);\n var stream:FileStream = new FileStream();\n stream.open(file, FileMode.READ)\n var strSql:String = stream.readUTFBytes(stream.bytesAvailable);\n NonQuery(strSql);\n}\n\npublic static function NonQuery(strSQL:String):void\n{\n var sqlConnection:SQLConnection = new SQLConnection();\n sqlConnection.open(File.applicationStorageDirectory.resolvePath(DBPATH);\n var sqlStatement:SQLStatement = new SQLStatement();\n sqlStatement.text = strSQL;\n sqlStatement.sqlConnection = sqlConnection;\n try\n {\n sqlStatement.execute();\n }\n catch (error:SQLError)\n {\n Alert.show(error.toString());\n }\n}\n</code></pre>\n\n<p>No errors are generated, however only <code>tRole</code> exists. It seems that it only looks at the first query (up to the semicolon- if I remove it, the query fails). Is there a way to call multiple queries in one statement?</p>\n"
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for TortoiseSVN?,"<p>Are there any really good tutorials explaining <a href=""http://svnbook.red-bean.com/en/1.8/svn.branchmerge.html"" rel=""nofollow"">branching and merging</a> with Apache Subversion? </p>\n\n<p>All the better if it's specific to TortoiseSVN client.</p>\n"
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,"<p>Has anyone got experience creating <strong>SQL-based ASP.NET</strong> site-map providers?</p>\n\n<p>I've got the default XML file <code>web.sitemap</code> working properly with my Menu and <strong>SiteMapPath</strong> controls, but I'll need a way for the users of my site to create and modify pages dynamically.</p>\n\n<p>I need to tie page viewing permissions into the standard <code>ASP.NET</code> membership system as well.</p>\n"
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,"<p>This is something I've pseudo-solved many times and never quite found a solution. That's stuck with me. The problem is to come up with a way to generate <code>N</code> colors, that are as distinguishable as possible where <code>N</code> is a parameter.</p>\n"
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applications,"<p>I have a little game written in C#. It uses a database as back-end. It's \na <a href=""http://en.wikipedia.org/wiki/Collectible_card_game"">trading card game</a>, and I wanted to implement the function of the cards as a script.</p>\n\n<p>What I mean is that I essentially have an interface, <code>ICard</code>, which a card class implements (<code>public class Card056 : ICard</code>) and which contains function that are called by the game.</p>\n\n<p>Now, to make the thing maintainable/moddable, I would like to have the class for each card as source code in the database and essentially compile it on first use. So when I have to add/change a card, I'll ju

In [34]:
# Group tags by id and join them
df_tags['Tag'] = df_tags['Tag'].astype(str)
grouped_tags = df_tags.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))
# Reset index for making simpler dataframe
grouped_tags.reset_index()
grouped_tags_final = pd.DataFrame({'Id':grouped_tags.index, 'Tags':grouped_tags.values})
# Drop unnecessary columns
df_questions.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)

# Merge questions and tags into one dataframe
df = df_questions.merge(grouped_tags_final, on='Id')

In [35]:
import nltk

# Filter out questions with a score lower than 5
new_df = df[df['Score']>5]

# Split tags in order to get a list of tags
new_df['Tags'] = new_df['Tags'].apply(lambda x: x.split())
all_tags = [item for sublist in new_df['Tags'].values for item in sublist]

flat_list = [item for sublist in new_df['Tags'].values for item in sublist]

keywords = nltk.FreqDist(flat_list)
keywords = nltk.FreqDist(keywords)

# Get most frequent tags
frequencies_words = keywords.most_common(25)
tags_features = [word[0] for word in frequencies_words]
# Drop unnecessary columns at this point
new_df.drop(columns=['Id', 'Score'], inplace=True)
print(tags_features)

['c#', 'java', 'javascript', 'android', 'python', 'c++', 'php', 'jquery', '.net', 'ios', 'html', 'css', 'c', 'iphone', 'objective-c', 'ruby-on-rails', 'sql', 'asp.net', 'mysql', 'ruby', 'r', 'git', 'asp.net-mvc', 'linux', 'sql-server']


In [36]:
def most_common(tags):
    """Function to check if tag is in most common tag list"""
    tags_filtered = []
    for i in range(0, len(tags)):
        if tags[i] in tags_features:
            tags_filtered.append(tags[i])
    return tags_filtered

# Change Tags column into None for questions that don't have a most common tag
new_df['Tags'] = new_df['Tags'].apply(lambda x: most_common(x))
new_df['Tags'] = new_df['Tags'].apply(lambda x: x if len(x)>0 else None)

# Drop rows that contain None in Tags column
new_df.dropna(subset=['Tags'], inplace=True)
new_df.shape

(52418, 3)

In [37]:
from bs4 import BeautifulSoup
import lxml
import re

from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

# Filter out HTML
new_df['Body'] = new_df['Body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text()) 

token = ToktokTokenizer()
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

def removeStopWords(text):
    words = token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    return ' '.join(map(str, filtered))

def removePunctuation(text):
    punct = '!"$%&\'()*,./:;<=>?@[\\]^_`{|}~'
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in tags_features:
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))

def lemmatizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x.lower())
    return ' '.join(map(str, listLemma))


# Remove stopwords, punctuation and lemmatize for text in body
new_df['Body'] = new_df['Body'].apply(lambda x: removeStopWords(x))
new_df['Body'] = new_df['Body'].apply(lambda x: removePunctuation(x))
new_df['Body'] = new_df['Body'].apply(lambda x: lemmatizeWords(x))

# Remove stopwords, punctuation and lemmatize for title. Also weight title 3 times
new_df['Title'] = new_df['Title'].apply(lambda x: str(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: removePunctuation(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: removeStopWords(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: lemmatizeWords(x)) 
new_df['Title'] = new_df['Title'].apply(lambda x: ' '.join(x.split()*3))
new_df['Title']

2                                                                                                             aspnet site maps aspnet site maps aspnet site maps
4                         adding script functionality net applications adding script functionality net applications adding script functionality net applications
5                                                                         should i use nest class case should i use nest class case should i use nest class case
6                                                          homegrown consumption web service homegrown consumption web service homegrown consumption web service
7                                     deploying sql server databases test live deploying sql server databases test live deploying sql server databases test live
                                                                                   ...                                                                          
1262668                using lambd

In [38]:
############################### CONFIG
MAX_LEN = 225
TRAIN_BATCH_SIZE = 36
VALID_BATCH_SIZE = 36
EPOCHS = 5
LEARNING_RATE = 1e-05

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

In [39]:
################################ INIT
warnings.simplefilter("ignore")
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [40]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [41]:
new_df.head()

,Title,Body,Tags
2,aspnet site maps aspnet site maps aspnet site maps,has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,"[sql, asp.net]"
4,adding script functionality net applications adding script functionality net applications adding script functionality net applications,i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,"[c#, .net]"
5,should i use nest class case should i use nest class case should i use nest class case,i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue,[c++]
6,homegrown consumption web service homegrown consumption web service homegrown consumption web service,i write web service .net app i ready consume them i see numerous examples homegrown code consume service oppose use auto generate methods visual studio create add web reference is advantage,[.net]
7,deploying sql server databases test live deploying sql server databases test live deploying sql server databases test live,i wonder guy manage deployment database 2 sql servers specifically sql server 2005 now development live one as part buildscript standard windows batch even current complexity script might switch powershell later enterprise managermanagement studio express count would copy mdf file attach i always bite careful work binary data seem compatiblity issue even though development live run version server time or - give lack explain create table t-sql - something export exist database sql-scripts run target server if yes tool automatically dump give database sql queries run command line again enterprise managermanagement studio express count and lastly - give fact live database already contain data deployment may involve create table rather check difference structure alter table live ones instead may also need data verificationconversion exist field change now hear lot great stuff red gate products hobby project price bite steep so use automatically deploy sql server databases test live,[sql-server]


In [42]:
new_df['Combo'] = new_df['Title'] + ". " + new_df['Body']

In [43]:
new_df.head()

,Title,Body,Tags,Combo
2,aspnet site maps aspnet site maps aspnet site maps,has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,"[sql, asp.net]",aspnet site maps aspnet site maps aspnet site maps. has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well
4,adding script functionality net applications adding script functionality net applications adding script functionality net applications,i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,"[c#, .net]",adding script functionality net applications adding script functionality net applications adding script functionality net applications. i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language
5,should i use nest class case should i use nest class case should i use nest class case,i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue,[c++],should i use nest class case should i use nest class case should i use nest class case. i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue
6,homegrown consumption web service homegrown consumption web service homegrown consumption web service,i write web service .net app i ready consume them i see numerous examples homegrown code consume service oppose use auto generate methods 

In [44]:
############################### BINARIZATION
mlb = MultiLabelBinarizer()
tag_df = pd.DataFrame(mlb.fit_transform(new_df['Tags']), columns=mlb.classes_, index=new_df.index)
class_names = mlb.classes_

In [45]:
class_names

array(['.net', 'android', 'asp.net', 'asp.net-mvc', 'c', 'c#', 'c++',
       'css', 'git', 'html', 'ios', 'iphone', 'java', 'javascript',
       'jquery', 'linux', 'mysql', 'objective-c', 'php', 'python', 'r',
       'ruby', 'ruby-on-rails', 'sql', 'sql-server'], dtype=object)

In [46]:
len(class_names)

25

In [47]:
df = new_df
df.head()

,Title,Body,Tags,Combo
2,aspnet site maps aspnet site maps aspnet site maps,has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,"[sql, asp.net]",aspnet site maps aspnet site maps aspnet site maps. has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well
4,adding script functionality net applications adding script functionality net applications adding script functionality net applications,i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,"[c#, .net]",adding script functionality net applications adding script functionality net applications adding script functionality net applications. i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language
5,should i use nest class case should i use nest class case should i use nest class case,i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue,[c++],should i use nest class case should i use nest class case should i use nest class case. i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue
6,homegrown consumption web service homegrown consumption web service homegrown consumption web service,i write web service .net app i ready consume them i see numerous examples homegrown code consume service oppose use auto generate methods 

In [48]:
############################### DATAFRAME HOUSEKEEPING
df = df.join(tag_df)
df = df.drop(columns='Tags')
df.head()

,Title,Body,Combo,.net,android,asp.net,asp.net-mvc,c,c#,c++,css,git,html,ios,iphone,java,javascript,jquery,linux,mysql,objective-c,php,python,r,ruby,ruby-on-rails,sql,sql-server
2,aspnet site maps aspnet site maps aspnet site maps,has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,aspnet site maps aspnet site maps aspnet site maps. has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,adding script functionality net applications adding script functionality net applications adding script functionality net applications,i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,adding script functionality net applications adding script functionality net applications adding script functionality net applications. i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,should i use nest class case should i use nest class case should i use nest class case,i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue,should i use nest class case should i use nest class case should i use nest class case. i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue,0,0,0,0,0,0,1,0,0,0,0,0

In [49]:
len(df.columns)

28

In [50]:
df['target_list'] = df.iloc[:, 3:].values.tolist()

In [51]:
df.head()

,Title,Body,Combo,.net,android,asp.net,asp.net-mvc,c,c#,c++,css,git,html,ios,iphone,java,javascript,jquery,linux,mysql,objective-c,php,python,r,ruby,ruby-on-rails,sql,sql-server,target_list
2,aspnet site maps aspnet site maps aspnet site maps,has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,aspnet site maps aspnet site maps aspnet site maps. has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
4,adding script functionality net applications adding script functionality net applications adding script functionality net applications,i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,adding script functionality net applications adding script functionality net applications adding script functionality net applications. i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,should i use nest class case should i use nest class case should i use nest class case,i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue,should i use nest class case should i use nest class case should i use nest class case. i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate

In [52]:
df = df.drop(df.columns[3:-1], axis=1)
df.head()

,Title,Body,Combo,target_list
2,aspnet site maps aspnet site maps aspnet site maps,has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,aspnet site maps aspnet site maps aspnet site maps. has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
4,adding script functionality net applications adding script functionality net applications adding script functionality net applications,i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,adding script functionality net applications adding script functionality net applications adding script functionality net applications. i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
5,should i use nest class case should i use nest class case should i use nest class case,i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue,should i use nest class case should i use nest class case should i use nest class case. i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
6,homegrown consumption web service h

In [53]:
df = df.drop(df.columns[3:4], axis=1)
df.head()

,Title,Body,Combo
2,aspnet site maps aspnet site maps aspnet site maps,has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well,aspnet site maps aspnet site maps aspnet site maps. has anyone get experience create sql-based aspnet site-map providers i get default xml file websitemap work properly menu sitemappath control i need way users site create modify page dynamically i need tie page view permissions standard aspnet membership system well
4,adding script functionality net applications adding script functionality net applications adding script functionality net applications,i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language,adding script functionality net applications adding script functionality net applications adding script functionality net applications. i little game write c# it use database back-end it trade card game i want implement function card script what i mean i essentially interface icard card class implement public class card056 icard contain function call game now make thing maintainablemoddable i would like class card source code database essentially compile first use so i addchange card i add database tell application refresh without need assembly deployment especially since would talk 1 assembly per card mean hundreds assemblies is possible register class source file instantiate etc icard cards current new mygamecardlibrarycard056 cards current onenterplay ref currentgamestate the language c# extra bonus possible write script net language
5,should i use nest class case should i use nest class case should i use nest class case,i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue,should i use nest class case should i use nest class case should i use nest class case. i work collection class use video playback record i one main class act like public interface methods like play stop pause record etc then i workhorse class video decode video encode i learn existence nest class c++ i curious know programmers think use them i little wary really sure benefitsdrawbacks seem accord book i read use case mine the book suggest scenario like mine good solution would nest workhorse class inside interface class separate file class client mean use avoid possible name conflict i know justifications nested class new concept me just want see programmers think issue
6,homegrown consumption web service homegrown consumption web service homegrown consumption web service,i write web service .net app i ready consume them i see numerous examples homegrown code consume service oppose use auto generate methods visual studio create add web reference is

In [54]:
# Define X, y
X1 = new_df['Body']
X2 = new_df['Title']
y = new_df['Tags']
print(len(X1), len(X2), len(y))
# Define multilabel binarizer
multilabel_binarizer = MultiLabelBinarizer()
y_bin = multilabel_binarizer.fit_transform(y)

52418 52418 52418


In [55]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, hamming_loss
from scipy.sparse import hstack
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for a text column
def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # CLS token embeddings

# Get BERT embeddings for 'Title' and 'Body'
title_embeddings = get_bert_embeddings(X1.tolist())
body_embeddings = get_bert_embeddings(X2.tolist())

# Combine title and body embeddings using hstack
X = hstack([title_embeddings, body_embeddings])


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 82446385152 bytes.

In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size = 0.2, random_state = 0)

In [ ]:
class_names = multilabel_binarizer.classes_
print(len(class_names))
class_names

In [ ]:
# Using Classifier Chains
from sklearn.multioutput import ClassifierChain
import numpy as np


chains = [ClassifierChain(svc, order='random', random_state=i)
          for i in range(10)]

for chain in chains:
    chain.fit(X_train, y_train)

Y_pred_chains = np.array([chain.predict(X_test) for chain in
                          chains])

Y_pred_ensemble = Y_pred_chains.mean(axis=0)
ensemble_accuracy_score = accuracy_score(y_test, Y_pred_ensemble >= .5)
ensemble_recall_score = recall_score(y_test, Y_pred_ensemble >= .5, average='weighted')
ensemble_precision_score = precision_score(y_test, Y_pred_ensemble >= .5, average='weighted')
ensemble_f1_score = f1_score(y_pred, Y_pred_ensemble >= .5, average='weighted')
hamm = hamming_loss(Y_pred_ensemble >= .5, y_test)*100
print(ensemble_accuracy_score, ensemble_recall_score, ensemble_precision_score, ensemble_f1_score, hamm)
